In [1]:
import tensorflow as tf

In [2]:
nums = tf.random.shuffle(tf.range(5))

In [3]:
nums

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([1, 2, 3, 0, 4])>

In [4]:
sort_nums = tf.sort(nums)

In [5]:
sort_nums

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 1, 2, 3, 4])>

In [7]:
sort_nums_desc = tf.sort(nums,direction='DESCENDING')

In [8]:
sort_nums_desc # 逆序排列

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([4, 3, 2, 1, 0])>

In [9]:
sort_index = tf.argsort(nums)

In [10]:
sort_index # 升序排列时的对应下标

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([3, 0, 1, 2, 4])>

In [11]:
a = tf.random.uniform([3,3],maxval=10,dtype='int32')

In [12]:
a

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1, 6, 6],
       [3, 4, 3],
       [2, 6, 3]])>

In [17]:
sort_a = tf.sort(a,axis=1) # 表示在行方向上作升序排列

In [18]:
sort_a

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1, 6, 6],
       [3, 3, 4],
       [2, 3, 6]])>

In [19]:
# top_k操作

In [20]:
a = tf.nn.top_k(a,k=2) # 得到前n个最大数以及他们的下标

In [21]:
a

TopKV2(values=<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[6, 6],
       [4, 3],
       [6, 3]])>, indices=<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [1, 0],
       [1, 2]])>)

In [24]:
a.indices # 下标

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [1, 0],
       [1, 2]])>

In [25]:
a.values # 前n个大的值

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[6, 6],
       [4, 3],
       [6, 3]])>

In [26]:
# top_k accuracy操作

In [28]:
def accuracy(output,target,top_k = (1,)):
    '''
    output 预测值 [B,N]
    target 实际值 [B]
    top_k = 选取多少的容错率
    '''
    maxk = max(top_k)
    banch_size = target.shape[0]
    
    pred = tf.math.top_k(output,maxk).indices # 预测值前几的下标
    pred = tf.transpose(pred,perm=[1,0]) # 转置
    target_ = tf.broadcast_to(target,pred.shape) # 目标值扩增
    correct = tf.cast(tf.equal(target_,pred),dtype='float32') # 损失值
    
    res = []
    for k in top_k:
        correct_k = tf.reshape(correct[:k],[-1]) # 转置变形
        correct_k = tf.reduce_sum(correct_k) # 计算总和
        acc = correct_k/banch_size # 计算不同k时的预测率
        res.append(acc) # 下标为K的多少，acc为该K下的预测率
    return res

In [29]:
# 测试

In [30]:
output = tf.random.normal([10,6]) # 10个数据集六类 
output = tf.math.softmax(output,axis=1) # 使数据在行上面总和为1
target = tf.random.uniform([10],maxval=6,dtype='int32') # 实际值

In [31]:
output

<tf.Tensor: shape=(10, 6), dtype=float32, numpy=
array([[0.1049338 , 0.23042232, 0.08390649, 0.34383217, 0.1475183 ,
        0.08938698],
       [0.04059754, 0.4173271 , 0.19048667, 0.10971481, 0.05885441,
        0.18301943],
       [0.51128125, 0.07474133, 0.06501567, 0.23634808, 0.07611562,
        0.03649811],
       [0.3057228 , 0.3913571 , 0.02825491, 0.15120953, 0.05588211,
        0.06757355],
       [0.09471017, 0.07560135, 0.6279899 , 0.05568954, 0.10929915,
        0.03670989],
       [0.01265294, 0.04774377, 0.1481491 , 0.31631544, 0.00610872,
        0.46903005],
       [0.02690466, 0.1267069 , 0.23347229, 0.15063033, 0.0761433 ,
        0.38614255],
       [0.0620532 , 0.17346114, 0.05251699, 0.1421211 , 0.29702148,
        0.27282608],
       [0.15273437, 0.0423807 , 0.16470389, 0.3476237 , 0.12361732,
        0.16893993],
       [0.02684665, 0.19015622, 0.05673401, 0.14258605, 0.28215685,
        0.30152023]], dtype=float32)>

In [32]:
target

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 5, 4, 0, 2, 5, 3, 3, 3, 4])>

In [33]:
# 最大化预测值

In [34]:
pred = tf.argmax(output,axis=1)

In [35]:
pred

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([3, 1, 0, 1, 2, 5, 5, 4, 3, 5], dtype=int64)>

In [36]:
# top_k 预测

In [37]:
res = accuracy(output,target,top_k=(1,2,3,4,5,6))

In [42]:
res

[<tf.Tensor: shape=(), dtype=float32, numpy=0.3>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.5>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.8>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1.0>]